# FMD Analysis Ver3

**在服务器中使用**
***
F->Fake News;

M->Main Stream Media;

D->Debunkers;
***

In [1]:
# 导入需要的库
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

## 数据预处理

*注：新冠疫情首批感染者出现在2019年12月，另外2023年5月5日，世界卫生组织宣布，新冠疫情不再构成“国际关注的突发公共卫生事件”。文件9_filtered.csv到27_filtered.csv包含从首批感染者出现开始，到世界卫生组织发表声明几个月之前的数据。*

将所有文件合并

In [2]:
# 将指定的X_filtered.csv文件合并
'''
def cat_file(file_num_list: list) -> pd.DataFrame:

    df_ls = []
    for i in file_num_list:
        f = pd.read_csv("../csv_filtered/" + str(i) + "_filtered.csv")
        df_ls.append(f)
        print(i)
    
    df_cat = pd.concat(df_ls, axis=0)
    df_cat.drop(['Unnamed: 0'], axis=1, inplace=True)
    return df_cat

# 合并从第9到27号文件
# 文件内部的数据是从时间新到旧排序的，为了按时间顺序拼接，将list反转
df = cat_file(list(range(9, 27+1))[::-1])
'''

'\ndef cat_file(file_num_list: list) -> pd.DataFrame:\n\n    df_ls = []\n    for i in file_num_list:\n        f = pd.read_csv("../csv_filtered/" + str(i) + "_filtered.csv")\n        df_ls.append(f)\n        print(i)\n    \n    df_cat = pd.concat(df_ls, axis=0)\n    df_cat.drop([\'Unnamed: 0\'], axis=1, inplace=True)\n    return df_cat\n\n# 合并从第9到27号文件\n# 文件内部的数据是从时间新到旧排序的，为了按时间顺序拼接，将list反转\ndf = cat_file(list(range(9, 27+1))[::-1])\n'

In [3]:
# 保存文件
#df.to_csv("data/full_data.csv", index=False)

In [4]:
df = pd.read_csv("data/full_data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808339 entries, 0 to 3808338
Data columns (total 83 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   id                                     int64  
 1   conversation_id                        int64  
 2   referenced_tweets.replied_to.id        float64
 3   referenced_tweets.retweeted.id         float64
 4   referenced_tweets.quoted.id            float64
 5   author_id                              int64  
 6   in_reply_to_user_id                    float64
 7   in_reply_to_username                   object 
 8   retweeted_user_id                      float64
 9   retweeted_username                     object 
 10  quoted_user_id                         float64
 11  quoted_username                        object 
 12  created_at                             object 
 13  text                                   object 
 14  lang                                   object 
 15

普通用户列表：将follower数小于500的用户视为普通用户

In [6]:
def gen_normal_user_list(df: pd.DataFrame) -> pd.DataFrame:
    # 将df的列名转换成字典：{col_index: col_key}
    cols = dict(enumerate(df.columns))
    # 调换键和值：{col_key: col_index}
    swapped_cols = {v:k for k,v in cols.items()}
    # 丢弃不需要的列
    cols_drop = list(range(0, 1+swapped_cols['withheld.country_codes'])) + \
            list(range(swapped_cols['context_annotations'], 1+swapped_cols['attachments.poll_ids'])) + \
            list(range(swapped_cols['author.withheld.country_codes'], len(swapped_cols)))
    cols_drop = [cols[i] for i in cols_drop]
    df_drop_cols = df.drop(cols_drop, axis=1)
    # 在'author.username'去重，得到所有用户的列表
    user_all = df_drop_cols.drop_duplicates(subset=['author.username'], keep='first')
    # 丢弃follower达到500的用户
    idx_500 = {i for i in user_all.index if user_all['author.public_metrics.followers_count'][i] >= 500}
    return user_all.drop(idx_500, axis=0)

In [14]:
# user_norm = gen_normal_user_list(df)
# user_norm.to_csv("data/user_norm.csv", index=False)

In [8]:
user_norm = pd.read_csv("data/user_norm.csv")
user_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824093 entries, 0 to 824092
Data columns (total 28 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   entities.annotations                   753527 non-null  object 
 1   entities.cashtags                      317 non-null     object 
 2   entities.hashtags                      382086 non-null  object 
 3   entities.mentions                      241700 non-null  object 
 4   entities.urls                          594518 non-null  object 
 5   author.id                              824093 non-null  int64  
 6   author.created_at                      824093 non-null  object 
 7   author.username                        824093 non-null  object 
 8   author.name                            823962 non-null  object 
 9   author.description                     587239 non-null  object 
 10  author.entities.description.cashtags   569 non-null     

读取媒体列表：虚假信息媒体、主流媒体、辟谣媒体

In [9]:
# 虚假信息媒体（fake news）
fn_df = pd.read_csv("data/FakeNewsDomain_from_iffy.news_23.09.06.csv")
fn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Domain                 1481 non-null   object 
 1   Site Rank              1481 non-null   int64  
 2   Year online            1435 non-null   float64
 3   Name                   1481 non-null   object 
 4   Media Bias/Fact Check  1481 non-null   object 
 5   MBFC Fact              1480 non-null   object 
 6   MBFC Bias              1481 non-null   object 
 7   MisinfoMe              1339 non-null   float64
 8   Wikipedia              748 non-null    object 
 9   Lang                   1480 non-null   object 
 10  URL                    1481 non-null   object 
 11  MBFC cred              1481 non-null   object 
 12  Unnamed: 12            0 non-null      float64
 13  Unnamed: 13            0 non-null      float64
dtypes: float64(4), int64(1), object(9)
memory usage: 162.1+ 

In [10]:
# 主流媒体（mainstream）
ms_df = pd.read_csv("data/high_credibility_websites_CoVaxxy.csv")
ms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   site_name       27 non-null     object
 1   site            27 non-null     object
 2   twitter_handle  27 non-null     object
dtypes: object(3)
memory usage: 776.0+ bytes


In [11]:
# 辟谣媒体（debunking）
db_df = pd.read_csv("data/(Merge)debunking_fact-checking_sites.csv")
db_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             45 non-null     object
 1   domain           41 non-null     object
 2   twitter_account  33 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


## 数据分析

### 计数每名普通用户分享三种媒体的数量

In [12]:
# 给每种媒体的数据建好一个空栏
user_norm['fake_news_share'] = ''
user_norm['mainstream_share'] = ''
user_norm['debunking_share'] = ''

# 三种媒体的网站域名和推特账号
fn_domain = set(fn_df['Domain'].dropna())

ms_domain = set(ms_df['site'].dropna())
ms_account = set(ms_df['twitter_handle'].dropna())

db_domain = set(db_df['domain'].dropna())
db_account = set(db_df['twitter_account'].dropna())

In [ ]:
# 使用numpy array加快查找速度
df_author = np.array(df['author.username'])
n = 70 * 10000
skip = 0
begin = 0
end = 0

for i in user_norm.index:
    # 跳过前n个用户
    if skip < n:
        skip += 1
        continue
    # 挑出i号用户发过的所有帖子的下标
    indexes = np.argwhere(df_author == user_norm['author.username'][i]).flatten().tolist()
    # 计数虚假信息媒体推文
    user_norm['fake_news_share'][i] = \
        ( \
            {r for r in indexes if re.search('|'.join(fn_domain), string=df['text'][r])} \
        ).__len__()
    # 计数主流媒体推文
    user_norm['mainstream_share'][i] = \
        ( \
            {r for r in indexes if re.search('|'.join(ms_domain), string=df['text'][r])} \
            #+ {s for s in indexes if str(df['quoted_username'][s]) in ms_account} \
            #+ {t for t in indexes if str(df['retweeted_username'][t]) in ms_account} \
        ).__len__()
    # 计数辟谣媒体推文
    user_norm['debunking_share'][i] = \
        ( \
            {r for r in indexes if re.search('|'.join(db_domain), string=df['text'][r])} \
            #+ {s for s in indexes if str(df['quoted_username'][s]) in db_account} \
            #+ {t for t in indexes if str(df['retweeted_username'][t]) in db_account} \
        ).__len__()
    # 暂存数据
        # 开始计数工作，并记录开始的用户下标
    if skip == n:
        begin = i
        print("begin = %d" % (begin))
        skip = np.inf
        continue
        # 每隔10000用户保存一次
    n += 1
    if (n % 1000) == 0:
        print("i = %d, n = %d" % (i, n))
        if (n % 10000) == 0:
            end = i
            print("package = %d, begin = %d, end = %d" % (n // 10000, begin, end))
            save = user_norm[:][begin : end]
            save.to_csv("TmpOutPut/" + str(n // 10000) + "_user.csv")
            begin = i
else:
    end = -1
    save = user_norm[:][begin : end]
    save.to_csv("TmpOutPut/" + str(n // 10000 + 1) + "_user.csv")
    print(n // 10000 + 1)

begin = 700000
i = 701000, n = 701000


### 计算“假新闻偏信分数”

定义“假新闻偏信分数”为：

fnbs = (fns - mss - dbs) / (fns + mss + dbs + 0.001)

fnbs: fake_news_believe_score

fns: fake_news_share

mss: mainstream_share

dbs: debunking_share

In [13]:
# 合并多个X_user.csv文件
def cat_csv_files(file_name_list: list, output_file_name: str) -> None:
    # 把文件加进一个列表
    df_ls = []
    for s in file_name_list:
        df = pd.read_csv(s)
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
        df_ls.append(df)
    # 合并
    df_cat = pd.concat(df_ls, axis=0)
    df_cat.to_csv(output_file_name, index=False)

In [14]:
# 合并多个X_user.csv文件
file_name_list = ["TmpOutPut/" + str(i) + "_user.csv" for i in range(1, 83+1)][::-1]
#cat_csv_files(file_name_list, "data/normal_user.csv")

In [2]:
user_norm = pd.read_csv("data/normal_user.csv")
user_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824092 entries, 0 to 824091
Data columns (total 31 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   entities.annotations                   753527 non-null  object 
 1   entities.cashtags                      317 non-null     object 
 2   entities.hashtags                      382086 non-null  object 
 3   entities.mentions                      241700 non-null  object 
 4   entities.urls                          594518 non-null  object 
 5   author.id                              824092 non-null  int64  
 6   author.created_at                      824092 non-null  object 
 7   author.username                        824092 non-null  object 
 8   author.name                            823961 non-null  object 
 9   author.description                     587238 non-null  object 
 10  author.entities.description.cashtags   569 non-null     

In [3]:
# 计算“假新闻偏信分数”
#'''
user_norm['fake_news_believe_score'] = ''
b = 0.001

n = 0
for i in user_norm.index:
    user_norm['fake_news_believe_score'][i] = \
        (user_norm['fake_news_share'][i] - user_norm['mainstream_share'][i] - user_norm['debunking_share'][i]) / \
        (user_norm['fake_news_share'][i] + user_norm['mainstream_share'][i] + user_norm['debunking_share'][i] + b)
    n += 1
    if n % 1000 == 0:
        print(n)
#'''

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [4]:
# 统计“假新闻偏信分数”为正值、零值和负值的用户有多少
#'''
score_z = [i for i in user_norm.index if user_norm['fake_news_believe_score'][i] == 0].__len__()
score_p = [i for i in user_norm.index if user_norm['fake_news_believe_score'][i] > 0].__len__()
score_n = [i for i in user_norm.index if user_norm['fake_news_believe_score'][i] < 0].__len__()

print(score_p, score_z, score_n)
#'''

416 823325 351


In [8]:
r = [i for i in user_norm.index if user_norm['fake_news_share'][i] > 0].__len__()
s = [i for i in user_norm.index if user_norm['mainstream_share'][i] > 0].__len__()
t = [i for i in user_norm.index if user_norm['debunking_share'][i] > 0].__len__()
print(r, s, t)

424 192 167
